In [1]:
from llmservice.generation_engine import GenerationEngine
from llmservice.schemas import GenerationRequest
from pydantic import BaseModel, Field
from typing import List


In [2]:

# Define schema for medical extraction
class MedicalRecord(BaseModel):
    patient_name: str = Field(description="Patient's full name")
    age: int = Field(ge=0, le=150, description="Patient age")
    conditions: List[str] = Field(description="Medical conditions")
    medications: List[str] = Field(description="Current medications")
    allergies: List[str] = Field(default_factory=list, description="Known allergies")

# Create generation request with schema
engine = GenerationEngine(model_name="gpt-4o-mini")

request = GenerationRequest(
    user_prompt="John Smith, 52 years old, has type 2 diabetes and high blood pressure. He takes metformin and lisinopril. Allergic to penicillin.",
    system_prompt="Extract medical information",
    response_schema=MedicalRecord,  # Pass the schema here
    reasoning_effort="low",  # Low reasoning for better format compliance
    model="gpt-4o-mini"
)

# Generate with structured output
result = engine.generate_output(request)

if result.success:
    # Parse the JSON response to Pydantic model
    import json
    data = json.loads(result.content)
    record = MedicalRecord(**data)
    
    print(f"Patient: {record.patient_name}, Age: {record.age}")
    print(f"Conditions: {', '.join(record.conditions)}")
    print(f"Medications: {', '.join(record.medications)}")
    print(f"Allergies: {', '.join(record.allergies)}")

Patient: John Smith, Age: 52
Conditions: type 2 diabetes, high blood pressure
Medications: metformin, lisinopril
Allergies: penicillin
